In [28]:
import pandas as pd 
import json
from arcgis.features import FeatureLayer
import requests
import time

# API Call to Louisville Open Data. 

| Steps      | Description | Notes     |
| :---        |    :----   |          :--- |
| 1. Open Data API call      | API call for restaurant inspection scores | Works   |
| 2. Clean   | Remove un-needed data        | drop cols and sort data. Also ned to only keep the most recent scores      |

In [29]:
# url = 'https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/FoodServiceData/FeatureServer/0'

# batch_size = 1000  # Number of records to retrieve per batch
# offset = 0  # Initial offset value
# data_list = []
# # ​Create the feature layer object
# feature_layer = FeatureLayer(url)

# while True:
#     # Query the feature layer with pagination
#     query_result = feature_layer.query(where='1=1', out_fields='*', return_geometry=False, result_offset=offset, result_record_count=batch_size)
    
#     # Retrieve the features from the query result
#     features = query_result.features
    
#     # Process the data for the current batch
#     for feature in features:
#         data_list.append(feature.attributes)
    
#     # Break the loop if the response is empty or the desired number of records is reached
#     if len(features) == 0 or len(data_list) >= 1000:
#         break
    
#     # Increment the offset by the batch size
#     offset += batch_size
# # Create a DataFrame from the data list
# df = pd.DataFrame(data_list)

# df.to_csv('../json_files/local_working.csv')

In [30]:
df = pd.read_csv('../json_files/local_working.csv')

Verify the data comes in correct

In [31]:
df.head()

,Unnamed: 0,EstablishmentID,InspectionID,Ins_TypeDesc,EstablishmentName,PlaceName,Address,Address2,City,State,Zip,TypeDescription,InspectionDate,score,Grade,NameSearch,Intersection,ObjectId
0,0,29956,1768584,REGULAR,CVS #6211,NaN,3721 LEXINGTON RD,NaN,LOUISVILLE,KY,40207,PRE-PACKAGED RETAIL,2023-06-30 00:00:00,100,NaN,CVS #6211,NaN,1
1,1,29967,1786484,REGULAR,NEIGHBORHOOD FOOD MART,NaN,542 LAMPTON ST,NaN,LOUISVILLE,KY,40203,SUPERMARKET WITH PROCESSING,2023-09-05 00:00:00,100,NaN,NEIGHBORHOOD FOOD MART,NaN,2
2,2,29979,1731827,REGULAR,CIRCLE K #4700075,NaN,1630 ARTHUR ST,NaN,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-01-27 00:00:00,95,A,CIRCLE K #4700075,NaN,3
3,3,29979,1731828,REGULAR,CIRCLE K #4700075,NaN,1630 ARTHUR ST,NaN,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-01-27 00:00:00,97,NaN,CIRCLE K #4700075,NaN,4
4,4,29979,1766455,REGULAR,CIRCLE K #4700075,NaN,1630 ARTHUR ST,NaN,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-06-21 00:00:00,98,A,CIRCLE K #4700075,NaN,5


Drops the extra cols we dont need. 

In [32]:
cols_drop = ['EstablishmentID', 'InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
df.drop(cols_drop, axis=1, inplace=True)
df.head(2)

,Unnamed: 0,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
0,0,REGULAR,CVS #6211,3721 LEXINGTON RD,LOUISVILLE,KY,40207,2023-06-30 00:00:00,100,NaN,1
1,1,REGULAR,NEIGHBORHOOD FOOD MART,542 LAMPTON ST,LOUISVILLE,KY,40203,2023-09-05 00:00:00,100,NaN,2


In [33]:
df = df[df['score'] != 0]

In [34]:
df.shape

(9969, 11)

Converting InspectionDate to date time

In [35]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])
# sorting inspection dates
df.sort_values('InspectionDate', ascending=False, inplace=True)
# dropping duplicate rest based on its first occurrence
df.drop_duplicates(subset='EstablishmentName', keep='first', inplace=True)
df.head(2)

,Unnamed: 0,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
105,105,REGULAR,IROQUOIS HIGH SCHOOL,4615 TAYLOR BLVD,LOUISVILLE,KY,40215,2023-09-15,100,A,106
389,389,REGULAR,CHRISTIAN HEALTH CENTER,920 S 4TH ST,LOUISVILLE,KY,40203,2023-09-15,99,A,390


In [36]:
df.shape

(4107, 11)

In [37]:
df = df.sort_values(by='score', ascending=True)
df.head(3)

,Unnamed: 0,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
8878,8878,REGULAR,EXPRESSWAY FOOD MART,1420 W BROADWAY,LOUISVILLE,KY,40203,2023-07-27,76,C,8879
656,656,REGULAR,A CHAU GROCERY,5107 S 3RD ST,LOUISVILLE,KY,40214,2023-09-14,83,NaN,657
7543,7543,SURVEY,BEEF O'BRADY'S,3101 S 2ND ST,LOUISVILLE,KY,40208,2023-08-25,88,NaN,7544


In [38]:
df.to_json('../json_files/health.json')

# Yelp API 

Imports my API key for yelp. 

In [39]:
from akeys import api_key

In [40]:
# api_key
# endpoint = "businesses/search"
# url = f"https://api.yelp.com/v3/{endpoint}"
# headers = {
#     "Authorization": f"Bearer {api_key}"
# }
# params = {
#     "term": "restaurants",
#     "location": "Louisville, KY",
#     "limit": 40  # Set the desired limit per request (maximum is 50)
# }

# restaurant_data = []  # List to store the extracted data

# offset = 0  # Initial offset value
# results_per_request = params["limit"]  # Results per request (50)
# total_results = float("inf")  # Initialize total_results to an arbitrary high value

# while offset < total_results:
#     params["offset"] = offset  # Set the offset parameter
    
#     response = requests.get(url, headers=headers, params=params)
    
#     if response.status_code != 200:
#         print(f"Error: {response.status_code} - {response.text}")
#         break
    
#     data = response.json()

#     if offset == 0:
#         total_results = data["total"]
    
#     businesses = data["businesses"]
    
#     for business in businesses:
#         name = business.get("name")
#         rating = business.get("rating")
#         review_count = business.get("review_count")
#         price = business.get("price")
#         address = ", ".join(business.get("location", {}).get("display_address", []))
#         restaurant_data.append({
#             "Name": name,
#             "Rating": rating,
#             "Review Count": review_count,
#             "Price": price,
#             "Address": address
#         })
#     offset += results_per_request  
#     time.sleep(2)  

# # Create a DataFrame from the extracted data
# df2 = pd.DataFrame(restaurant_data)

# # Print the DataFrame
# df2.head(3)

In [41]:
# df2.shape
# df2.to_csv('yelp.csv')

In [42]:
df2 = pd.read_csv('../json_files/yelp.csv')

In [43]:
# # cols_drop = ['InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
# # df.drop(cols_drop, axis=1, inplace=True)
# df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])
# # Grouping by EstablishmentID and aggregating the necessary columns
# df = df.groupby("EstablishmentID").agg({
#     "EstablishmentName": "first",
#     "Address": "first",
#     "score": lambda x: list(x)
# })

# # Sorting the establishments based on the count of low scores
# df["low_score_count"] = df["score"].apply(lambda x: sum(score < 75 for score in x))
# df = df.sort_values("low_score_count", ascending=False)

# # Creating the final DataFrame with the desired columns
# df = df[["EstablishmentName", "Address", "score"]]

# # Printing the final DataFrame
# df.head(10)

In [44]:
df2.tail(20)

,Unnamed: 0,Name,Rating,Review Count,Price,Address
980,980,Good Belly,4.5,3,NaN,"Louisville, KY 40202"
981,981,Penn Station East Coast Subs,2.5,20,$,"300 W Woodlawn Ave, Ste 102, Louisville, KY 40214"
982,982,Get In Your Belly Deli,4.0,8,NaN,"Louisville, KY 40202"
983,983,Chung King Palace,2.5,30,$,"110 E Market St, Louisville, KY 40202"
984,984,Louisville Hookah Lounge and Kufiya Cafe,5.0,3,$,"5400 DelMaria Way, Louisville, KY 40291"
985,985,China Express,5.0,1,NaN,"4946 Poplar Level Rd, Louisville, KY 40219"
986,986,Mi Preferida 3,4.5,2,NaN,"4752 Bardstown Rd, Louisville, KY 40218"
987,987,McDonald's,2.0,31,$,"2209 Taylorsville Rd, Louisville, KY 40205"
988,988,Genji Sushi Bars Louisville,4.0,1,NaN,"4944 Shelbyville Rd, Whole Foods Market, Louis..."
989,989,Buffalo Wild Wings,2.0,134,$$,"4600 Shelbyville Rd, St. Matthews, KY 40207"


In [45]:
df2['Rating'].unique()

array([4.5, 4. , 5. , 3.5, 3. , 0. , 2.5, 1. , 2. , 1.5])

In [46]:
# df2 = df2.sort_values(by="Rating", ascending=True)
# df2 = df2.loc[df["Rating"] != 0.0]
# df2 = df2.reset_index(drop=True)
# df2['Rating'] = (df2['Rating']*20).astype(int)
# df2.head(3)
# df2.to_json('../json_files/yelp.json')

In [47]:
# Sort df2 by the "Rating" column in ascending order
df2 = df2.sort_values(by="Rating", ascending=True)

# Remove rows where the "Rating" column value is 0.0
df2 = df2.loc[df2["Rating"] != 0.0]

# Reset the index of df2
df2 = df2.reset_index(drop=True)

# Multiply the "Rating" column by 20 and convert it to an integer
df2['Rating'] = (df2['Rating'] * 20).astype(int)

# Display the first 3 rows of df2
df2.head(3)

# Save df2 to a JSON file
# df2.to_json('../json_files/yelp.json')


,Unnamed: 0,Name,Rating,Review Count,Price,Address
0,301,Afrokanza Lounge,20,1,NaN,"1578 Bardstown Rd, Louisville, KY 40205"
1,460,Griyo Caribbean Restaurant,20,3,NaN,"519 W Oak St, Louisville, KY 40203"
2,682,Steak ’n Shake,30,101,$,"4545 Outer Lp, Louisville, KY 40219"


In [48]:
df2['Address'] = df2['Address'].str.split(',').str[0]
# df2['Rating'] = (df2['Rating']*20).astype(int)
df2.head(3)

,Unnamed: 0,Name,Rating,Review Count,Price,Address
0,301,Afrokanza Lounge,20,1,NaN,1578 Bardstown Rd
1,460,Griyo Caribbean Restaurant,20,3,NaN,519 W Oak St
2,682,Steak ’n Shake,30,101,$,4545 Outer Lp


In [49]:
df['Address'] = df['Address'].str.lower().str.strip()
df2['Address'] = df2['Address'].str.lower().str.strip()

In [50]:
df2.shape

(968, 6)

In [51]:
df2['Rating'].unique()

array([ 20,  30,  40,  50,  60,  70,  80,  90, 100])

In [52]:
merged_df = df.merge(df2, on='Address', how='left')
merged_df = merged_df.sort_values('Rating', ascending=True)
merged_df = merged_df[['EstablishmentName', 'Address', 'score', 'Rating', 'Review Count']]
merged_df = merged_df.rename(columns={'Review Count': 'ReviewCount'})
merged_df.head(3)

,EstablishmentName,Address,score,Rating,ReviewCount
711,GRIYO CARIBBEAN RESTAURANT,519 w oak st,96,20.0,3.0
122,AFROKANZA LOUNGE,1578 bardstown rd,94,20.0,1.0
2892,POD DAVIDSON,2100 s floyd st,100,30.0,5.0


In [53]:
processed_data = merged_df.to_json(orient='records')

In [54]:
# Save the processed data as JSON to a file
with open('../json_files/processed_data.json', 'w') as file:
    json.dump(processed_data, file)